In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.sparse as sp
import itertools
from torch.utils.data import random_split
import wandb
import os
import pandas as pd
import warnings
import torchvision


In [2]:
warnings.filterwarnings('ignore')

In [82]:
assert os.getenv('WANDB_API_KEY'), 'You must set the WANDB_API_KEY environment variable'

In [3]:
##Login wandb client
wandb.login()

wandb: Currently logged in as: amaboh (avri). Use `wandb login --relogin` to force relogin


True

In [4]:
from torchvision.datasets import CIFAR10


In [5]:
from torchvision.datasets import CIFAR10

train_data = CIFAR10(root="../data/train/",
                     train=True,
                     download=True)

100%|██████████████████████████████████████████████████| 170498071/170498071 [13:30<00:00, 210264.84it/s]


Extracting ../data/train/cifar-10-python.tar.gz to ../data/train/


In [6]:
print(train_data)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../data/train/
    Split: Train


In [7]:
print(len(train_data))

50000


In [8]:
print(train_data.data.shape)

(50000, 32, 32, 3)


In [89]:
#print(train_data.targets)

In [10]:
print(train_data.classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [11]:
print(train_data.class_to_idx)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [12]:
print(type(train_data[0]))

<class 'tuple'>


In [13]:
print(len(train_data[0]))

2


In [14]:
data, label = train_data[0]

In [15]:
print(type(data))

<class 'PIL.Image.Image'>


In [16]:
print(type(label))

<class 'int'>


In [17]:
print(train_data.classes[label])

frog


In [18]:
test_data = CIFAR10(root="../data/test/",
                    train=False,
                    download=True)

100%|██████████████████████████████████████████████████| 170498071/170498071 [04:45<00:00, 596415.50it/s]


Extracting ../data/test/cifar-10-python.tar.gz to ../data/test/


In [19]:
print(test_data)

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ../data/test/
    Split: Test


In [20]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=(0.4914, 0.4822, 0.4465),
    std=(-.2023, 0.1994, 0.2010))
])
    
train_data = CIFAR10(
                    root="../data/test/",
                    train=True,
                    download=True,
                    transform=train_transforms
                    )

Files already downloaded and verified


In [21]:
print(train_data)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../data/test/
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(-0.2023, 0.1994, 0.201))
           )


In [22]:
print(train_data.transforms)

StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(-0.2023, 0.1994, 0.201))
           )


In [23]:
data, label = train_data[0]

In [24]:
print(type(data))

<class 'torch.Tensor'>


In [25]:
print(data.size)

<built-in method size of Tensor object at 0x16a39f4c0>


In [26]:
print(data)

tensor([[[ 2.4291e+00,  2.4291e+00,  2.4291e+00,  ...,  2.4291e+00,
           2.4291e+00,  2.4291e+00],
         [-5.1744e-01, -6.3375e-01, -6.3375e-01,  ...,  1.5955e+00,
           1.2854e+00,  2.4291e+00],
         [ 1.2226e-01,  4.4721e-02, -1.4913e-01,  ...,  2.4291e+00,
           2.1189e+00,  2.4291e+00],
         ...,
         [ 1.3435e+00, -9.2452e-01, -1.1765e+00,  ..., -1.4673e+00,
          -1.8356e+00,  2.4291e+00],
         [ 1.3435e+00, -6.7252e-01, -2.2667e-01,  ..., -1.4673e+00,
          -1.6030e+00,  2.4291e+00],
         [ 5.4873e-01, -1.1378e+00, -1.1036e-01,  ..., -9.2452e-01,
          -1.0602e+00,  2.4291e+00]],

        [[-2.4183e+00, -2.4183e+00, -2.4183e+00,  ..., -2.4183e+00,
          -2.4183e+00, -2.4183e+00],
         [ 4.0101e-02,  1.7777e-01,  1.3844e-01,  ..., -1.5136e+00,
          -1.1989e+00, -2.4183e+00],
         [-7.8591e-01, -6.8757e-01, -5.3024e-01,  ..., -2.4183e+00,
          -2.0249e+00, -2.4183e+00],
         ...,
         [-1.8479e+00,  4

In [27]:
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
    (0.4914, 0.4822, 0.4465),
    (0.2023, 0.1994, 0.2010))
])

test_data = torchvision.datasets.CIFAR10(
    root= "../data/test",
    train=False,
    transform=test_transforms
)

print(test_data)

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ../data/test
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )


In [50]:
train_set, val_set = random_split(
                      train_data,
                      [40000, 10000])

trainloader = torch.utils.data.DataLoader(
                train_set,
                batch_size=16,
                shuffle=True
                )

In [29]:
data_batch, labels_batch = next(iter(trainloader))
print(data_batch.size())


torch.Size([16, 3, 32, 32])


In [30]:
print(labels_batch.size())

torch.Size([16])


In [51]:
testloader = torch.utils.data.DataLoader(
              test_data,
              batch_size = 16,
              shuffle=False
            )

valloader = torch.utils.data.DataLoader(
                    val_set,
                    batch_size=16,
                    shuffle=True)


In [52]:
from torchvision import models

try: 
    vgg16 = models.vgg16(weights='DEFAULT')
except Exception as e:
    print(f'Exception: {e}')

In [33]:
print(vgg16.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [34]:
torch.hub.list(
      'nvidia/DeepLearningExamples:torchhub')

Using cache found in /Users/ama/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub


['nvidia_convnets_processing_utils',
 'nvidia_efficientnet',
 'nvidia_efficientnet_b0',
 'nvidia_efficientnet_b4',
 'nvidia_efficientnet_widese_b0',
 'nvidia_efficientnet_widese_b4',
 'nvidia_fastpitch',
 'nvidia_gpunet',
 'nvidia_hifigan',
 'nvidia_resneXt',
 'nvidia_resnet50',
 'nvidia_resnext101_32x4d',
 'nvidia_se_resnext101_32x4d',
 'nvidia_ssd',
 'nvidia_ssd_processing_utils',
 'nvidia_tacotron2',
 'nvidia_textprocessing_utils',
 'nvidia_tft',
 'nvidia_tft_data_utils',
 'nvidia_tts_utils',
 'nvidia_waveglow']

In [35]:
import torch.nn as nn
import torch.nn.functional as F

class SmallNet(nn.Module):
    
    def __init__(self):
        super(SmallNet, self).__init__()
        self.fc1 = nn.Linear(2048, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)
        
    def forward(self, x):
        x = x.view(-1, 2048)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [36]:
smallnet = SmallNet()
print(SmallNet)

<class '__main__.SmallNet'>


In [37]:
input = torch.rand(2048)
output = smallnet(input)

In [38]:
from torch import nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),
                        (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1,
                   int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [39]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [56]:
N_EPOCHS = 10
lr=0.001
momentum = 0.9

In [75]:
from torch import optim
from torch import nn

model = LeNet5().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                     lr=lr)

In [76]:
run = wandb.init(
    # Set the project where this run will be logged
    project="Effective_MLOPs_CICD_CV",
    # Track hyperparameters and run metadata
    tags=['challengerV1'],
    config={
        "learning_rate": lr,
        "epochs": N_EPOCHS,
        "optimizer":"Adam"
    })

epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
test accuracy,▁
train loss,█▅▄▃▃▂▂▁▁▁
val loss,█▆▅▄▃▂▂▂▁▁
epoch,9
test accuracy,0.2526
train loss,1.20114
val loss,1.24148


In [77]:

for epoch in range(N_EPOCHS):

    # Training
    train_loss = 0.0
    model.train()  # Set the model to training mode
    for inputs, labels in trainloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Log training loss
    avg_train_loss = train_loss / len(trainloader)
    wandb.log({"epoch": epoch, "train loss": avg_train_loss})

    # Validation
    val_loss = 0.0
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No gradient computation
        for inputs, labels in valloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

    # Log validation loss
    avg_val_loss = val_loss / len(valloader)
    wandb.log({"epoch": epoch, "val loss": avg_val_loss})

    print(
      "Epoch: {} Train Loss: {} Val Loss: {}".format(
                  epoch, avg_train_loss, avg_val_loss))

Epoch: 0 Train Loss: 1.6986474369049072 Val Loss: 1.5111891620635987
Epoch: 1 Train Loss: 1.4647838483572007 Val Loss: 1.4184259377479553
Epoch: 2 Train Loss: 1.3743178421974183 Val Loss: 1.4236837748527527
Epoch: 3 Train Loss: 1.322928121304512 Val Loss: 1.3030801772117615
Epoch: 4 Train Loss: 1.2774642323493957 Val Loss: 1.3171323024749755
Epoch: 5 Train Loss: 1.2456189161539077 Val Loss: 1.2583462390899658
Epoch: 6 Train Loss: 1.2220693056941032 Val Loss: 1.2672287676811218
Epoch: 7 Train Loss: 1.1915092902779578 Val Loss: 1.2265211025238036
Epoch: 8 Train Loss: 1.171791988992691 Val Loss: 1.201218919658661
Epoch: 9 Train Loss: 1.1568260403513908 Val Loss: 1.1752229278564452


In [78]:
num_correct = 0.0
for x_test_batch, y_test_batch in testloader:
  model.eval() 
  y_test_batch = y_test_batch.to(device)
  x_test_batch = x_test_batch.to(device)
  y_pred_batch = model(x_test_batch) 
  _, predicted = torch.max(y_pred_batch, 1) 
  num_correct += (predicted ==
    y_test_batch).float().sum() 

accuracy = num_correct/(len(testloader) \
  *testloader.batch_size) 

print(len(testloader), testloader.batch_size)

print("Test Accuracy: {}".format(accuracy))
wandb.log({"test accuracy": accuracy})

625 16
Test Accuracy: 0.2517000138759613


In [85]:
import wandb.apis.reports as wr

PROJECT = 'Effective_MLOPs_CICD_CV'
ENTITY = 'avri'

report = wr.Report(
    entity=ENTITY,
    project=PROJECT,
    title = "Compare Runs",
    description="comparing runs of model"
)

In [86]:
pg = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT, "Run Comparison").set_filters_with_python_expr("Name in ['skilled-firefly-5', 'dulcet-dawn-6']")
    ],
    panels=[
        wr.RunComparer(diff_only='split', layout={'w': 24, 'h': 15}),
    ]
)

report.blocks = report.blocks[:1] + [pg] + report.blocks[1:]
report.save()

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


Report(project='Effective_MLOPs_CICD_CV', entity='avri', title='Compare Runs', description='comparing runs of model', width='readable', blocks=[PanelGrid(runsets=[Runset(entity='avri', project='Effective_MLOPs_CICD_CV', name='Run Comparison', query='', filters={'$or': [{'$and': [{'displayName': {'$in': ['skilled-firefly-5', 'dulcet-dawn-6']}}]}]}, order=['-CreatedTimestamp'])], panels=[RunComparer(diff_only='split')])])

In [87]:
report.url

'https://wandb.ai/avri/Effective_MLOPs_CICD_CV/reports/Compare-Runs--Vmlldzo2MzY4MzQy'

In [88]:
import pickle 

with open('model_pickle', 'wb') as f:
    pickle.dump(model, f)